In [3]:
!pip install faiss-cpu

!pip install gradio

In [4]:
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import gradio as gr

# Load embedding model
embed_model = SentenceTransformer("sentence-transformers/paraphrase-mpnet-base-v2")

# Sample Hinglish dataset
documents = [
    "Bhai, kal party hai. Tu aa raha hai na?",
    "Aaj cricket ka match bahut dhamakedar tha!",
    "Mujhe coding aur AI research bahut pasand hai.",
    "Kal ka traffic to hadd se zyada tha, bore ho gaya.",
]

# Compute embeddings
embeddings = embed_model.encode(documents, convert_to_numpy=True)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

def retrieve_and_generate(query):
    """Retrieves relevant Hinglish text and generates a response using LLM."""
    query_embedding = embed_model.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embedding, k=2)  # Retrieve top-2 similar docs
    context = "\n".join([documents[i] for i in I[0]])

    # Load optimized LLM (4-bit Quantized)
    model_name = "mistralai/Mistral-7B-Int4"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    quantization_config = BitsAndBytesConfig(load_in_4bit=True)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True
    )

    prompt = f"Context: {context}\nUser: {query}\nAI:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    output = model.generate(**inputs, max_new_tokens=20)  # Reduce token count for faster response
    response = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

    return response

# Gradio UI
demo = gr.Interface(
    fn=retrieve_and_generate,
    inputs=gr.Textbox(label="Ask something in Hinglish"),
    outputs=gr.Textbox(label="AI Response"),
    title="Hinglish RAG Chatbot (Optimized)"
)

demo.launch(share=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://79abaa9b2b6353a623.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
